In [1]:

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv('TEST.csv')

In [2]:
df

,windspeed,rainfall,area,yield,percentageofarea,darea,yielda,fgprice,dctrice
0,0,1.535972,2870.00,3.37,0.247070,709.091324,0.832626398,20.79,11135504
1,0,2.141071,491.00,4.36,0.244428,120.014173,1.065706303,20.79,2412279.37
2,7.116629346,5.809375,1330.88,4.42,0.312983,416.543187,1.383386094,20.79,10868283.84
3,67.08946747,3.908854,5767.00,4.65,0.567852,3274.800267,2.640510013,20.79,163091064.2
4,0,0.805469,4710.00,4.14,0.130472,614.521934,0.540153038,20.79,6260539.25
...,...,...,...,...,...,...,...,...,...
21016,0,0.522989,2936.00,0,0.264956,777.910078,0,14.32,0
21017,0,0.110000,156.00,2.85,0.101081,15.768650,0.288081111,14.32,59014.04
21018,10.51271508,0.225000,6376.00,6.88,0.131057,835.618326,0.901670967,14.32,9788183.48
21019,2.823089172,0.379861,2065.00,3.14,0.115129,237.741383,0.361505057,14.32,1116516.69


In [3]:
# Split the dataset into features and target variable
X = df.drop(['dctrice','fgprice'], axis=1)
y = df.drop(['darea','percentageofarea','yielda','windspeed','rainfall','area', 'yield', 'fgprice'], axis=1)


In [4]:
X.shape, y.shape

((21021, 7), (21021, 1))

In [5]:
# Training and Temporary Set (80-20 split)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Temporary set into a validation set and a test set using (50-50 split)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [6]:

X_train.shape, y_train.shape

((16816, 7), (16816, 1))

In [7]:
y_train.info()
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16816 entries, 13628 to 15795
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dctrice  16816 non-null  object
dtypes: object(1)
memory usage: 262.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16816 entries, 13628 to 15795
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   windspeed         16816 non-null  object 
 1   rainfall          16816 non-null  float64
 2   area              16816 non-null  float64
 3   yield             16705 non-null  object 
 4   percentageofarea  16816 non-null  float64
 5   darea             16816 non-null  float64
 6   yielda            16816 non-null  object 
dtypes: float64(4), object(3)
memory usage: 1.0+ MB


In [8]:
X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')
X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
y_train = pd.DataFrame(y_train).apply(pd.to_numeric, errors='coerce')
y_test = pd.DataFrame(y_test).apply(pd.to_numeric, errors='coerce')

In [9]:
X_train.shape, y_train.shape

((16816, 7), (16816, 1))

In [10]:
# Remove the rows with zero values from the training set and testing set
mask = X_train['windspeed'] != 0

# Apply the mask to X_train
X_train = X_train[mask]

# Apply the mask to y_train
y_train = y_train[mask]

In [11]:
# Remove the rows with zero values from the training set and testing set
mask =y_train['dctrice'] != 0

# Apply the mask to X_train
X_train = X_train[mask]

# Apply the mask to y_train
y_train = y_train[mask]

In [12]:
X_train.shape, y_train.shape

((6653, 7), (6653, 1))

In [13]:
# Null Remover

nan_rows = X_train[X_train.isna().any(axis=1)].index
X_train = X_train.drop(nan_rows)
y_train = y_train.drop(nan_rows)

print(np.where(pd.isnull(X_train)))

(array([], dtype=int64), array([], dtype=int64))


In [14]:
X_train.shape,y_train.shape

((6647, 7), (6647, 1))

In [15]:
# mask = X_train['windspeed'] != 0


In [16]:
nan_rows = X_test[X_test.isna().any(axis=1)].index
X_test = X_test.drop(nan_rows)
y_test = y_test.drop(nan_rows)

In [17]:
nan_rows = X_val[X_val.isna().any(axis=1)].index
X_val = X_val.drop(nan_rows)
y_val = y_val.drop(nan_rows)

In [18]:
# mask = X_test['windspeed'] != 0

# # Apply the mask to X_train
# X_test = X_test[mask]

# y_test = y_test[mask]


In [19]:
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(
    n_estimators=100, 
    max_depth=4, 
    min_samples_split=8, 
    min_samples_leaf=42, 
    max_leaf_nodes=100,
    random_state=42,
   
)


model.fit(X_train, y_train)

predictionsVal = model.predict(X_val)

# Evaluate the model

maev = mean_absolute_error(y_val, predictionsVal)
print(f'Mean Absolute Error (Validation): {maev}')


r2 = model.score(X_val, y_val)
print(f'R-squared (Validation): {r2}')


predictions = model.predict(X_test)

# Evaluate the model

mae = mean_absolute_error(y_test, predictions)
print(f'\nMean Absolute Error (Test): {mae}')

mae_percentage_test = (mae / y_test.values.mean()) * 100
print(f'Mean Absolute Error (Test, p.p.): {mae_percentage_test}%')

r2 = model.score(X_test, y_test)
print(f'R-squared (Test): {r2}')


C:\Users\Asus\AppData\Local\Temp\ipykernel_17496\1069868929.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Mean Absolute Error (Validation): 2570528.460497208
R-squared (Validation): 0.9274569473397258

Mean Absolute Error (Test): 2717312.902493148
Mean Absolute Error (Test, p.p.): 32.19321851449802%
R-squared (Test): 0.8855940511583511


In [20]:
print(['{:.2f}'.format(pred) for pred in predictions])

['722647.98', '722647.98', '722647.98', '722647.98', '3715825.68', '6394959.22', '722647.98', '7495016.22', '3715825.68', '3892070.33', '722647.98', '7495016.22', '2891551.37', '28491621.01', '7495016.22', '1345589.83', '722647.98', '722647.98', '105615011.71', '136678550.27', '7495016.22', '722647.98', '722647.98', '3892070.33', '722647.98', '722647.98', '722647.98', '2891551.37', '28919633.03', '62732046.82', '3746463.95', '722647.98', '3715825.68', '722647.98', '722647.98', '722647.98', '2863673.46', '722647.98', '3715825.68', '722647.98', '722647.98', '18622717.77', '722647.98', '722647.98', '722647.98', '18622717.77', '722647.98', '722647.98', '3182493.05', '722647.98', '722647.98', '722647.98', '722647.98', '722647.98', '722647.98', '3715825.68', '3182493.05', '722647.98', '7495016.22', '62508681.88', '3963979.33', '722647.98', '7252708.00', '3715825.68', '722647.98', '3182493.05', '7495016.22', '722647.98', '722647.98', '722647.98', '722647.98', '722647.98', '2891551.37', '28915

In [21]:
from sklearn.pipeline import make_pipeline
from joblib import dump

pipeline = make_pipeline(model)

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Save the pipeline
dump(pipeline, 'RFR.joblib')

c:\Users\Asus\anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


['RFR.joblib']

In [22]:
r2 = pipeline.score(X_test, y_test)
print(f'R-squared (Test): {r2}')

R-squared (Test): 0.8855940511583511
